In [18]:
# 필요 라이브러리 import
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from selenium import webdriver
import json
import ast
import numpy as np
import pandas as pd
import time

- 미니 프로젝트 주제 : 왓챠 사이트에서 개별 구매 top20의 영화코드를 통해 평균 별점, 보고싶어요, 왓챠피디아 사용자 평(limit = 5) 크롤링
- 추후에 사용자 평으로 텍스트 감정 분석을 통해 긍정, 부정, 중립 분류 가능, 별점, 보고싶어요 목록을 통한 비슷한 장르를 선호하는 사용자 유사도를 통해 추천 시스템 만들기
- 개선할 점 : 영화코드 자동화 필요, RPA와 매크로 접목 필요, 더 많은 왓챠피디아 사용자 평 크롤링, 클래스 명이 변경되었을 경우 대비

In [20]:
# 영화코드 가져오기
with open('watcha_top20.txt', 'r', encoding='utf-8') as f:
    movie_text = f.read()
    
movie_code = ast.literal_eval(movie_text) # json은 무조건 ""(큰 따옴표) ''는 딕셔너리 형식
movie_code

{'신명': 'mOPV37X',
 '너의 이름은': 'mOo0PkN',
 '릴리 슈슈의 모든 것': 'mBOk6gW',
 '기쿠지로의 여름': 'mBOkyj5',
 '드래곤 길들이기': 'm5DP44y',
 '해피엔드': 'mW9pL6K',
 '드래곤 길들이기 2': 'mWzvzZW',
 '야당': 'mdKpkjk',
 '괴물': 'mOopwPa',
 '파과': 'mWw3jVl',
 '날씨의 아이': 'm5nwoZ9',
 '서브스턴스': 'mdB7jbP',
 '드래곤 길들이기 1': 'm5eQ6M5',
 '드래곤 길들이기 3': 'm5XMlv1',
 '꽃다발 같은 사랑을 했다': 'm5ZlR7z',
 '리턴 투 센더': 'm5xJbJj',
 '민용': 'tPeWoDB',
 '러브레터': 'mQO8b5R',
 '더 울프 오브 월스트리트': 'm5DpxB5',
 '배드 가이즈': 'mW42GlY'}

In [21]:
url_list = []
for key, value in movie_code.items():
    watcha_url = 'https://watcha.com/contents/' + value
    url_list.append(watcha_url)

In [22]:
url_list

['https://watcha.com/contents/mOPV37X',
 'https://watcha.com/contents/mOo0PkN',
 'https://watcha.com/contents/mBOk6gW',
 'https://watcha.com/contents/mBOkyj5',
 'https://watcha.com/contents/m5DP44y',
 'https://watcha.com/contents/mW9pL6K',
 'https://watcha.com/contents/mWzvzZW',
 'https://watcha.com/contents/mdKpkjk',
 'https://watcha.com/contents/mOopwPa',
 'https://watcha.com/contents/mWw3jVl',
 'https://watcha.com/contents/m5nwoZ9',
 'https://watcha.com/contents/mdB7jbP',
 'https://watcha.com/contents/m5eQ6M5',
 'https://watcha.com/contents/m5XMlv1',
 'https://watcha.com/contents/m5ZlR7z',
 'https://watcha.com/contents/m5xJbJj',
 'https://watcha.com/contents/tPeWoDB',
 'https://watcha.com/contents/mQO8b5R',
 'https://watcha.com/contents/m5DpxB5',
 'https://watcha.com/contents/mW42GlY']

In [23]:
driver = webdriver.Chrome()

In [24]:
driver.implicitly_wait(3)

In [25]:
movie_name = list(movie_code.keys())

In [26]:
mean_star = []
watcha_count = []

for i in range(len(url_list)):
    driver.get(url_list[i])
    time.sleep(1)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    
    ms = soup.find_all('span', {'class':'SKjQD'})[0].text
    mean_star.append(ms)
    wc = soup.find_all('span', {'class':'SKjQD'})[1].text
    watcha_count.append(wc)

In [27]:
movie_name_list = []
user_name = []
user_star = []
user_content = []

for i in range(len(url_list)):
    driver.get(url_list[i])
    time.sleep(1)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    for j in range(5):
        movie_name_list.append(movie_name[i])
        un = soup.find_all('div', {'class':'AitgY MBvIw'})[j].text # 0~4
        user_name.append(un)
        us = len(soup.find_all('div', {'class':'s1RWj'})[j].find_all('svg'))
        user_star.append(us)
        uc = soup.find_all('div', {'data-select':'embed-title-text'})[j].text # 0~4
        user_content.append(uc)

In [28]:
columns1 = ['movie_name', 'mean_star', 'watcha_count']
columns2 = ['movie_name', 'user_name', 'user_star', 'user_content']

In [29]:
movie_df = pd.DataFrame(data = list(zip(movie_name, mean_star, watcha_count)), columns = columns1)
content_df = pd.DataFrame(data = list(zip(movie_name_list, user_name, user_star, user_content)), columns = columns2)

In [30]:
movie_df

,movie_name,mean_star,watcha_count
0,신명,2.2,741
1,너의 이름은,4.0,70.4%
2,릴리 슈슈의 모든 것,3.7,"99,443"
3,기쿠지로의 여름,3.7,"57,996"
4,드래곤 길들이기,3.9,"4,492"
5,해피엔드,3.9,"15,397"
6,드래곤 길들이기 2,3.6,"12,903"
7,야당,3.0,"1,966"
8,괴물,4.3,78.9%
9,파과,3.2,"6,837"


In [31]:
content_df

,movie_name,user_name,user_star,user_content
0,신명,다솜땅,3,어디까지 진실이고 어디까지가 사실일까? 소름돋는건 당연한거고... 비슷하게 풍겨나는...
1,신명,김덕수,3,조아요
2,신명,위대한 성운-(Nebula),3,허접했다가 좋았다가 하는걸 반복하는 복합적엔 영화넼ㅋ
3,신명,진입금지,3,"현실과 관련된 영화 내용을 떠나서, 그 기간에 그 예산으로 만든것치고는 정말 잘만든..."
4,신명,probohemia,5,이게 진짜라면
...,...,...,...,...
95,배드 가이즈,영화보는 강아지,4,진부한 하이스트에 진부한 메세지를 유니크하게 접근한다
96,배드 가이즈,펭돌,4,클리셰 파괴도 이젠 클리셰
97,배드 가이즈,함성규,4,살짝 어둡고 쾌활한 톤의 <주토피아> + <미션임파서블> 시리즈 + 그리고 <오션스...
98,배드 가이즈,ANImasiL,3,"겉은 야수여도 속은 평범한 인간이였다.\n뻔하지만, 매력적인 세계관과 캐릭터들이 영..."


In [32]:
top_list = pd.DataFrame(movie_name).reset_index()
top_list['top'] = top_list['index'] + 1
top_list = top_list.drop('index',axis=1)
top_list = top_list.rename(columns = {0:'movie_name'})

In [34]:
df = movie_df.merge(content_df, on='movie_name', how='outer')
df = df.merge(top_list, on='movie_name', how='outer')
df = df.sort_values('top')
df = df.reindex(['top', 'movie_name', 'mean_star', 'watcha_count', 'user_name', 'user_star', 'user_content'], axis = 1)
df.to_csv('watcha_top20_crawling.csv', encoding='utf-8', index=False)
df

,top,movie_name,mean_star,watcha_count,user_name,user_star,user_content
84,1,신명,2.2,741,probohemia,5,이게 진짜라면
81,1,신명,2.2,741,김덕수,3,조아요
82,1,신명,2.2,741,위대한 성운-(Nebula),3,허접했다가 좋았다가 하는걸 반복하는 복합적엔 영화넼ㅋ
80,1,신명,2.2,741,다솜땅,3,어디까지 진실이고 어디까지가 사실일까? 소름돋는건 당연한거고... 비슷하게 풍겨나는...
83,1,신명,2.2,741,진입금지,3,"현실과 관련된 영화 내용을 떠나서, 그 기간에 그 예산으로 만든것치고는 정말 잘만든..."
...,...,...,...,...,...,...,...
73,20,배드 가이즈,3.4,"2,497",ANImasiL,3,"겉은 야수여도 속은 평범한 인간이였다.\n뻔하지만, 매력적인 세계관과 캐릭터들이 영..."
71,20,배드 가이즈,3.4,"2,497",펭돌,4,클리셰 파괴도 이젠 클리셰
74,20,배드 가이즈,3.4,"2,497",똥맛된장애호가,4,매력있는 캐릭터들
72,20,배드 가이즈,3.4,"2,497",함성규,4,살짝 어둡고 쾌활한 톤의 <주토피아> + <미션임파서블> 시리즈 + 그리고 <오션스...
